In [2]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [4]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet...
- [1 files][  7.1 GiB/  7.1 GiB]                                                
Operation completed over 1 objects/7.1 GiB.                                      


In [5]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'
base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria1'] = 0
data['clase_binaria2'] = 0
data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [8]:
meses_train = [202006,202007,202008,202009,202010,202011,202012,
               202101,202102,202103,202104,202105,202106]

data = data[data['foto_mes'].isin(meses_train)]

In [12]:
df_subsampled = []

# Iterar sobre cada grupo de 'foto_mes' para realizar el submuestreo
for mes, group in data.groupby("foto_mes"):
    # Separamos características y target
    X = group.drop(columns="clase_binaria2")
    y = group["clase_binaria2"]

    # Aplicar RandomUnderSampler con una estrategia de 10:1
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_res, y_res = rus.fit_resample(X, y)

    # Reconstruimos el DataFrame submuestreado del grupo
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Añadir nuevamente la columna 'foto_mes'

    # Añadimos el DataFrame submuestreado a la lista
    df_subsampled.append(group_resampled)

# Concatenamos todos los grupos submuestreados en un único DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

In [13]:
# Verificar la proporción de clases en el DataFrame submuestreado
class_proportions = data["clase_binaria2"].value_counts(normalize=True)
class_proportions

clase_binaria2
0    0.909091
1    0.090909
Name: proportion, dtype: float64

In [15]:
output_file = 'competencia_02_fe_v01_undersampled.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [16]:
!gsutil cp /home/eanegrin/datasets/competencia_02_fe_v01_undersampled.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_02_fe_v01_undersampled.parquet...
- [1 files][336.7 MiB/336.7 MiB]                                                
Operation completed over 1 objects/336.7 MiB.                                    
